<a href="https://colab.research.google.com/github/Arjavjain100/TOS-Summarization/blob/main/Pegasus_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.9 MB/s eta 0:00:00


In [5]:
!git clone https://github.com/Arjavjain100/TOS-Summarization.git

Cloning into 'TOS-Summarization'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 96 (delta 3), reused 0 (delta 0), pack-reused 82
Unpacking objects: 100% (96/96), 649.73 KiB | 831.00 KiB/s, done.


# New section

In [3]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00


In [7]:
import pandas as pd

filename = "/content/TOS-Summarization/Dataset/all_v1_transpose.csv"

df = pd.read_csv(filename)
df = df[['original_text','reference_summary']]
df.rename(columns = {'original_text':'source', 'reference_summary':'target'}, inplace = True)
len(df)

446

In [8]:
X = df['source']
y = df['target']

In [9]:
df.head()

,source,target
0,welcome to the pokémon go video game services ...,hi.
1,by using our services you are agreeing to thes...,by playing this game you agree to these terms....
2,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...
3,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...
4,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...


In [21]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments,pipeline,PretrainedConfig
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [11]:
class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels['input_ids'])

In [12]:
def prepare_data(model_name, 
                 train_texts, train_labels, 
                 test_texts, test_labels):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True, max_length = 600)
    decodings = tokenizer(labels, truncation=True, padding=True, max_length = 256)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, test_dataset, tokenizer

In [13]:
def prepare_fine_tuning(model_name, tokenizer, train_dataset, test_dataset, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  if test_dataset is not None:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=2,              # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=1,    # batch size for evaluation, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      evaluation_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=100,                  # number of update steps before evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=test_dataset,           # evaluation dataset
      tokenizer=tokenizer
    )

  else:
    training_args = TrainingArguments(
      output_dir=output_dir,           # output directory
      num_train_epochs=2,              # total number of training epochs
      per_device_train_batch_size=1,   # batch size per device during training, can increase if memory allows
      save_steps=500,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
    )

    trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      tokenizer=tokenizer
    )

  return trainer

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [15]:
train_texts, train_labels = list(X_train), list(y_train)
test_texts, test_labels = list(X_test), list(y_test)

In [16]:
model_name = 'nsi319/legal-pegasus'
train_dataset,test_dataset, tokenizer = prepare_data(model_name, train_texts, train_labels,test_texts,test_labels)
trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset,test_dataset)
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,10.524200,10.290859
200,9.106800,9.143480
300,8.444600,8.352876
400,6.944700,4.548946
500,2.883200,0.629687
600,0.870300,0.521460
700,0.757200,0.498111


TrainOutput(global_step=712, training_loss=5.566175214360269, metrics={'train_runtime': 558.1254, 'train_samples_per_second': 1.276, 'train_steps_per_second': 1.276, 'total_flos': 1205448435302400.0, 'train_loss': 5.566175214360269, 'epoch': 2.0})

In [17]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.4977768361568451,
 'eval_runtime': 11.8308,
 'eval_samples_per_second': 7.607,
 'eval_steps_per_second': 7.607,
 'epoch': 2.0}

In [18]:
import os
if not os.path.exists('./ouput_model/'):
    os.makedirs('./ouput_model/')
trainer.model.save_pretrained("./ouput_model/")

Inference

In [19]:
type(trainer)

transformers.trainer.Trainer

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [22]:
config = PretrainedConfig.from_json_file('./ouput_model/config.json')

In [23]:
model = PegasusForConditionalGeneration.from_pretrained("./ouput_model/", config = config).to(device)

In [24]:
def summarize(text):
  input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=1024,truncation=True).to(device)
  summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=50,
                                  max_length=150,
                                  early_stopping=True)
  summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

  return summary

In [25]:
y_pred = X_test.apply(lambda x: summarize(x))

In [26]:
summary = pd.concat([y_test.to_frame(name="reference_summary"), y_pred.to_frame(name="generated_summary")], axis=1)

In [27]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from rouge import Rouge

In [29]:
rouge = Rouge()

In [30]:
rouge.get_scores(summary['generated_summary'], summary['reference_summary'],avg=True)

{'rouge-1': {'r': 0.4822443484730356,
  'p': 0.1853939901578475,
  'f': 0.25242519452522133},
 'rouge-2': {'r': 0.16867109845566294,
  'p': 0.053012838572288966,
  'f': 0.07423816621313986},
 'rouge-l': {'r': 0.4240135359966986,
  'p': 0.1587215180478183,
  'f': 0.21734083297274054}}